# Spatial panel regression in stata using the xsmle package

## Initial setup

In [1]:
clear all
set more off

## Load weights matrix.dta (note the first column)

In [2]:
use weights.dta, clear

## Create spmat object and row normalize the weights matrix

In [3]:
spmat dta W m*, normalize(row)

## Summarize the weights matrix

In [4]:
spmat summarize W
spmat summarize W, links
spmat summarize W, links detail



Summary of spatial-weighting object W
--------------------------------------
        Matrix |          Description
---------------+----------------------
    Dimensions |            278 x 278
     Stored as |            278 x 278
 Values        |
           min |                    0
         min>0 |                   .1
          mean |             .0035971
           max |                    1
--------------------------------------


Summary of spatial-weighting object W
--------------------------------------
        Matrix |          Description
---------------+----------------------
    Dimensions |            278 x 278
     Stored as |            278 x 278
 Links         |
         total |                 1476
           min |                    1
          mean |             5.309353
           max |                   10
--------------------------------------


Summary of spatial-weighting object W
--------------------------------------
        Matrix |          Description
---

## Load panel dataset

In [5]:
use paneldata.dta, clear

## Define global parameters

In [6]:
global id mun
global t year
global ylist v1
global xlist v2 v3

## Declare that the data is in long-form panel shape

In [7]:
sort $id $t
xtset $id $t



       panel variable:  mun (strongly balanced)
        time variable:  year, 1 to 7
                delta:  1 unit


## Descriptive statistics

In [8]:
xtdescribe
xtsum $id $t $ylist $xlist



     mun:  1, 2, ..., 278                                    n =        278
    year:  1, 2, ..., 7                                      T =          7
           Delta(year) = 1 unit
           Span(year)  = 7 periods
           (mun*year uniquely identifies each observation)

Distribution of T_i:   min      5%     25%       50%       75%     95%     max
                         7       7       7         7         7       7       7

     Freq.  Percent    Cum. |  Pattern
 ---------------------------+---------
      278    100.00  100.00 |  1111111
 ---------------------------+---------
      278    100.00         |  XXXXXXX


Variable         |      Mean   Std. Dev.       Min        Max |    Observations
-----------------+--------------------------------------------+----------------
mun      overall |     139.5    80.2718          1        278 |     N =    1946
         between |              80.3959          1        278 |     n =     278
         within  |                    0    

## Pooled OLS model with information criteria: AIC and BIC

In [9]:
reg $ylist $xlist
estat ic



      Source |       SS           df       MS      Number of obs   =     1,946
-------------+----------------------------------   F(2, 1943)      =    564.95
       Model |  70.1274404         2  35.0637202   Prob > F        =    0.0000
    Residual |   120.59246     1,943  .062065085   R-squared       =    0.3677
-------------+----------------------------------   Adj R-squared   =    0.3670
       Total |  190.719901     1,945  .098056504   Root MSE        =    .24913

------------------------------------------------------------------------------
          v1 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
          v2 |   -.253897   .0160932   -15.78   0.000    -.2854588   -.2223352
          v3 |    1.39407   .1032402    13.50   0.000     1.191597    1.596543
       _cons |  -2.178912   .3292605    -6.62   0.000    -2.824653   -1.533171
-------------------------------------------------

## Spatial Durbin model (SDM) with random effects

In [10]:
xsmle v1 v2 v3, wmat(W) model(sdm) re vce(cluster mun) nolog
estat ic




SDM with random-effects                              Number of obs =      1946

Group variable: mun                               Number of groups =       278
Time variable: year                                   Panel length =         7

R-sq:    within  = 0.0005
         between = 0.4490
         overall = 0.3397

Log-pseudolikelihood =   434.3975
                                  (Std. Err. adjusted for 278 clusters in mun)
------------------------------------------------------------------------------
             |               Robust
          v1 |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
Main         |
          v2 |  -.4084206   .0737295    -5.54   0.000    -.5529277   -.2639134
          v3 |   .1381157   .3682051     0.38   0.708     -.583553    .8597844
       _cons |   1.002876   .9124532     1.10   0.272    -.7854994    2.791251
-------------+---------------------------

## SDM without direct, indirect and total effects

In [12]:
xsmle v1 v2 v3, wmat(W) model(sdm) noeffects nolog
estat ic




SDM with random-effects                              Number of obs =      1946

Group variable: mun                               Number of groups =       278
Time variable: year                                   Panel length =         7

R-sq:    within  = 0.0005
         between = 0.4490
         overall = 0.3397

Log-likelihood =   434.3975
------------------------------------------------------------------------------
          v1 |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
Main         |
          v2 |  -.4084206   .0446939    -9.14   0.000     -.496019   -.3208221
          v3 |   .1381157   .1959201     0.70   0.481    -.2458806     .522112
       _cons |   1.002876    .702365     1.43   0.153    -.3737342    2.379486
-------------+----------------------------------------------------------------
Wx           |
          v2 |   .1658575   .0606777     2.73   0.006     .0469315  

## SAR with spatial and time fixed-effects

In [13]:
xsmle v1 v2 v3, wmat(W) model(sar) fe type(both) vce(cluster mun) nolog
estat ic



SAR with spatial and time fixed-effects              Number of obs =      1946

Group variable: mun                               Number of groups =       278
Time variable: year                                   Panel length =         7

R-sq:    within  = 0.0120
         between = 0.4894
         overall = 0.3428

Mean of fixed-effects =  4.0282

Log-pseudolikelihood =   964.7944
                                  (Std. Err. adjusted for 278 clusters in mun)
------------------------------------------------------------------------------
             |               Robust
          v1 |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
Main         |
          v2 |   .0545333   .0507805     1.07   0.283    -.0449947    .1540613
          v3 |  -.9947755   .6513873    -1.53   0.127    -2.271471    .2819202
-------------+----------------------------------------------------------------
Spatial  